In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists
from demres.common.helper_functions import *
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *
from pprint import pprint
from IPython.display import display

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Process raw CSV files

In [5]:
# create_pegmed()

In [21]:
# create_pegprod()

In [22]:
# create_prescriptions()

In [23]:
# create_consultations()

In [24]:
# create_clinicals()

In [25]:
# create_tests()

In [26]:
# create_referrals()

In [27]:
# create_immunisations()

In [28]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, female gender, insomnia) used in the logistic regression*

In [6]:
all_encounters = get_all_encounters()

/Users/zurfarosa/demres/common/process_entries.py:156: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_encounters = pd.concat([consultations,medcoded_entries],ignore_index=True)


In [7]:
all_entries = get_all_entries(all_encounters)

/Users/zurfarosa/demres/common/process_entries.py:166: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_entries = pd.concat([all_encounters,prescriptions],ignore_index=True)


KeyboardInterrupt: 

In [ ]:
pt_features = create_pt_features()

In [ ]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [89]:
pt_features = add_data_start_and_end_dates(all_encounters,pt_features)

calculating latest_sysdate
calculating earliest_sysdate
resampling all_encounters - may take some time...
locating converted codes
choosing most appropriate measure of data_start
removing patients without any events
There are 14 patients without any events


In [90]:
pt_features.to_csv('data/processed_data/pt_features.csv',index=False)

In [8]:
pt_features = pd.read_csv('data/processed_data/pt_features.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [9]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start
74217,10185491,21,491,1,2004-12-14,True,6578.0,2006-11-13,2006-11-13
1282,4040005,30,5,1,NaT,False,NaN,2010-08-05,1997-01-31
82381,5989569,20,569,0,NaT,False,NaN,2013-03-29,2004-12-31
52611,9175349,29,349,0,2011-03-18,True,1916.0,2013-04-23,1999-10-12
75874,2626503,30,503,0,2012-05-04,True,6578.0,2013-05-17,2007-08-14


In [10]:
len(pt_features)

94500

In [12]:
len(pt_features[pt_features['isCase']==True])

46572

In [13]:
len(pt_features[pt_features['isCase']==False])

47928

In [91]:
#Remove specific dementias (e.g. FTLD, DLB) from pt_features, but save this to a different CSV file
pt_features_avoid_specific_dementia = avoid_specific_dementia_subtypes(pt_features) 
pt_features_avoid_specific_dementia.to_csv('data/processed_data/pt_features_avoid_specific_dementia.csv',index=False)

removing cases where final dementia subtype is a specific, non-Alzheimer, non-VaD dementia
Number of cases removed:  6442
Number of patients (cases and controls) 88058


In [14]:
# for window in [sd.exposure_windows[1]]:
#     printbold(window['name'])  
#     pt_features = pd.read_csv('data/processed_data/pt_features_avoid_specific_dementia.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = match_cases_and_controls(pt_features,window)
#     pt_features.to_csv('data/processed_data/pt_features_avoid_specific_dementia_'+ window['name'] +'.csv',index=False)


**10_to_5**

All cases 40130
Number of cases with 10 years of data 15973
Number of cases without 10 years of data (to be discarded): 24157
Number of controls 47928
27251  cases being removed as unmatchable
35049  controls being removed as unmatchable
62300  total patients being removed as unmatchable


## Add derived variables to pt_features 
*e.g. insomnia count, history of stroke, consultation count*

In [15]:
medcoded_entries = pd.read_hdf('data/processed_data/hdf/medcoded_entries.hdf')

In [16]:
prescriptions = pd.read_hdf('data/processed_data/hdf/prescriptions.hdf')

In [17]:
pt_features = pd.read_csv('data/processed_data/pt_features_avoid_specific_dementia_'+ sd.exposure_windows[1]['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [20]:
create_pdd_for_each_drug(prescriptions,pt_features,sd.exposure_windows[1])

In [22]:
for window in [sd.exposure_windows[1]]:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/processed_data/pt_features_avoid_specific_dementia_' + window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,prescriptions,window,codelists.all_codelists)
    pt_features = create_PDD_columns_for_each_pt(pt_features,window,[codelists.insomnia['medications']],prescriptions)
    pt_features = create_quantiles_and_booleans(pt_features)
#     display(pt_features.sample(2))    
    pt_features.to_csv('data/processed_data/pt_features_avoid_specific_dementia_'+ window['name'] +'.csv',index=False)